In [0]:
import dlt
from pyspark.sql.functions import current_timestamp
import requests

CATALOG = "bronze"  

# API Source → Bronze
@dlt.table(name=f"{CATALOG}.bronze_api_data", comment="Bronze layer from API")
def bronze_api_data():
    url = "https://www.abc.com/data"
    data = requests.get(url).json()
    return spark.createDataFrame(data).withColumn("ingestion_date", current_timestamp())


# ADLS Source → Bronze
@dlt.table(name=f"{CATALOG}.bronze_adls_data", comment="Bronze layer from ADLS")
def bronze_adls_data():
    return (
        spark.read.format("parquet")
        .load("abfss://container@storageaccount.dfs.core.windows.net/folder/")
        .withColumn("ingestion_date", current_timestamp())
    )


# Volumes / Local File → Bronze
@dlt.table(name=f"{CATALOG}.bronze_volumes_data", comment="Bronze layer from local volumes")
def bronze_volumes_data():
    return (
        spark.read.format("csv")
        .option("header", "true")
        .load("/mnt/volumes/data.csv")  #can be CSV/XML/JSON
        .withColumn("ingestion_date", current_timestamp())
    )
